In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [1]:
from langchain.document_loaders import TextLoader
# text to write a local file



#taken from https://www.theverge.com/2023/3/14/23639313/google-ai-language-model-palm-api-challenge-openai


text = """Google opens up its AI language model PaLM to challenge OpenAI and GPT-3 / The search giant is launching an API for its large language model PaLM. Businesses and developers will be able to build on the system to create custom chatbots and much more.
By James Vincent, a senior reporter who has covered AI, robotics, and more for eight years at The Verge.

Google logo with colorful shapes
Illustration: The Verge
Google is offering developers access to one of its most advanced AI language models: PaLM. The search giant is launching an API for PaLM alongside a number of AI enterprise tools it says will help businesses “generate text, images, code, videos, audio, and more from simple natural language prompts.”

PaLM is a large language model, or LLM, similar to the GPT series created by OpenAI or Meta’s LLaMA family of models. Google first announced PaLM in April 2022. Like other LLMs, PaLM is a flexible system that can potentially carry out all sorts of text generation and editing tasks. You could train PaLM to be a conversational chatbot like ChatGPT, for example, or you could use it for tasks like summarizing text or even writing code. (It’s similar to features Google also announced today for its Workspace apps like Google Docs and Gmail.)

In order to make it easier for developers to train PaLM to carry out specific tasks, Google is launching a new app alongside the PaLM API called MakerSuite. “With MakerSuite, you’ll be able to iterate on prompts, augment your dataset with synthetic data, and easily tune custom models,” said the company in a press release. Google says this sort of fine-tuning, which is necessary to create a consumer-friendly AI system, can even be done in a browser, with the computationally intensive work of training and deployment handled by Google Cloud.

In addition to launching the PaLM API, Google is also expanding support for generative AI in its Vertex AI platform, which is designed to help businesses train and deploy machine learning models. It says Vertex will have access to more models built by Google Research and its AI subsidiary DeepMind and will also be able to tap into open-source and third-party systems in the future.

And finally, Google is also launching a new platform called Generative AI App Builder. The company says this platform “allows developers to quickly ship new experiences including bots, chat interfaces, custom search engines, digital assistants, and more. Developers have API access to Google’s foundation models and can use out-of-the-box templates to jumpstart the creation of gen apps in minutes or hours.”

All of this may sound a bit dry and enterprise-focused — and it is! — but it’s also a big push by Google to put more AI tools in the hands of more companies. Specifically, access to PaLM will give businesses the opportunity to build new AI language services the same way OpenAI’s language APIs led to an explosion of startups. In other words, the PaLM API is another engine that will fuel new AI developments in 2023.
"""


with open("my_file.txt", "w") as file:
    file.write(text)
    
    
loader = TextLoader("my_file.txt")
docs_from_file = loader.load()

print(len(docs_from_file))

1


#### characterTextSplitter

In [2]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=20)


docs = text_splitter.split_documents(docs_from_file)
print(len(docs))



Created a chunk of size 354, which is longer than the specified 200
Created a chunk of size 357, which is longer than the specified 200
Created a chunk of size 541, which is longer than the specified 200
Created a chunk of size 544, which is longer than the specified 200
Created a chunk of size 383, which is longer than the specified 200
Created a chunk of size 408, which is longer than the specified 200


7


In [4]:
from langchain.embeddings import OpenAIEmbeddings


embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

In [5]:
from langchain.vectorstores import DeepLake

my_activeloop_ord_id = "Ali"
my_activeloop_dataset_name = "lanchain_course_indexers_retrievers"


dataset_path = f"hub://{my_activeloop_ord_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)


db.add_documents(docs)

Your Deep Lake dataset has been successfully created!


Creating 7 embeddings in 1 batches of size 7:: 100%|█████████████████████████████████████████████████████| 1/1 [00:12<00:00, 12.88s/it]

Dataset(path='hub://Ali/lanchain_course_indexers_retrievers', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
   text       text      (7, 1)      str     None   
 metadata     json      (7, 1)      str     None   
 embedding  embedding  (7, 1536)  float32   None   
    id        text      (7, 1)      str     None   


['6ee448e0-3c92-11ef-8bda-36ea86592e58',
 '6ee44afc-3c92-11ef-8bda-36ea86592e58',
 '6ee44b38-3c92-11ef-8bda-36ea86592e58',
 '6ee44b60-3c92-11ef-8bda-36ea86592e58',
 '6ee44b7e-3c92-11ef-8bda-36ea86592e58',
 '6ee44ba6-3c92-11ef-8bda-36ea86592e58',
 '6ee44bce-3c92-11ef-8bda-36ea86592e58']

In [14]:
# create retriever from db

retriever = db.as_retriever()

In [21]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
# text the author about this issue

# create a retrieval chain
qa_chain = RetrievalQA.from_chain_type(
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0),
chain_type="stuff",
retriever=retriever
)


In [22]:
query = "How Google plans to challenge OpenAI?"
response = qa_chain.run(query)
print(response)

Google plans to challenge OpenAI by opening up its AI language model PaLM through an API. This will allow businesses and developers to build custom chatbots and other applications using the system. Additionally, Google is launching a platform called Generative AI App Builder, which will enable developers to quickly create new experiences such as bots, chat interfaces, digital assistants, and more. By providing access to PaLM and expanding support for generative AI in its Vertex AI platform, Google aims to put more AI tools in the hands of companies and fuel new AI developments in 2023.
